# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [1]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


Ft9fLQe6uzz8vko2NRMO99IocMrseyzT31DbItuE


In [3]:
import cohere

co = cohere.ClientV2()

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response)



id='677a6ee5-8a24-41d3-82b8-3a3ba5cb1397' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0)) logprobs=None


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [4]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

{'paciente': {'nombre': 'María González', 'edad': 45},
 'fecha_admision': '2023-08-05',
 'sintomas': ['fatiga crónica', 'dolores musculares'],
 'diagnostico': 'fibromialgia',
 'tratamiento': ['fisioterapia', 'medicamentos analgésicos']}

In [5]:
#####

In [5]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [6]:
# your code here

prompt = f"""Extraer del texto delimitado por triples acentos graves las siguientes entidades: /
Paciente, Fecha de admisión, Síntomas, Diagnóstico, Tratamiento recomendado, y proporcionar la respuesta en string con formato de tipo json como el siguiente ejemplo:/ 
{{
  "paciente": {{
    "nombre": "María González",
    "edad": 45
  }},
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}}
```{text_to_analize}```
"""

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": prompt}],
)

llm_response = response.message.content[0].text
print(llm_response)

{
  "paciente": {
    "nombre": "Sofía López",
    "edad": 28
  },
  "fecha_admision": "2023-04-03",
  "sintomas": [
    "fiebre alta",
    "tos persistente"
  ],
  "diagnostico": "neumonía",
  "tratamiento": [
    "antibióticos",
    "reposo"
  ]
}


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [7]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."

In [8]:
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Agrega un contacto al diccionario.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto."
                    },
                    "phone": {
                        "type": "string",
                        "description": "Número de teléfono del contacto."
                    },
                    "email": {
                        "type": "string",
                        "description": "Correo electrónico del contacto."
                    },
                },
                "required": ["name", "phone", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Obtiene información específica de un contacto por su nombre, como su número de teléfono o correo electrónico.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto."
                    },
                },
                "required": ["name"]
            }
        }
    }
]

map_funciones = {
    "add_contact": add_contact,
    "get_information": get_information,
} 

In [64]:
mensaje_sistema = """
Eres un asistente que puede administrar contactos utilizando herramientas. 
Si te preguntan por información de un contacto, utiliza la herramienta `get_information` 
para recuperar los datos relacionados. Si te piden agregar un contacto, usa `add_contact`.
Utilizar los nombre tal cual como son ingresados por el usuario, por ejemplo:
Q: Cual es el Email de Joaquin Lopez?" 
tool_plan: Voy a utilizar la herramienta get_information para obtener el email de Joaquin Lopez.
"""

prompt = "Cual es el Email de Joaquin Lopez?"

messages = [
    {"role": "system", "content": mensaje_sistema},
    {"role": "user", "content": prompt},
]


In [65]:
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages,
                   tools=tools)
print (response)

print("Plan:")
print(response.message.tool_plan, "\n")
print("Llamar a:")
for tc in response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")
    
# Agrego el historial de chat
messages.append(
    {
        "role": "assistant",
        "tool_calls": response.message.tool_calls,
        "tool_plan": response.message.tool_plan,
    }
)


id='1b02e6bc-6989-48df-be46-ee085f22970b' finish_reason='TOOL_CALL' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=[ToolCallV2(id='get_information_045th5qw12s1', type='function', function=ToolCallV2Function(name='get_information', arguments='{"name":"Joaquin Lopez"}'))], tool_plan='Voy a utilizar la herramienta get_information para obtener el email de Joaquin Lopez.', content=None, citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=179.0, output_tokens=26.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=965.0, output_tokens=60.0)) logprobs=None
Plan:
Voy a utilizar la herramienta get_information para obtener el email de Joaquin Lopez. 

Llamar a:
Tool name: get_information | Parameters: {"name":"Joaquin Lopez"}


In [66]:
# Itero sobre las function calling generadas por el modelo

for tc in response.message.tool_calls:
    print(tc.function.arguments)
    
    # llamo a la función recomendada por el modelo usando los parametros recomendados
    salida_funcion = map_funciones[tc.function.name](**json.loads(tc.function.arguments))

    print( salida_funcion)

    if isinstance(salida_funcion, dict):
        content = json.dumps(salida_funcion)
    else:
        content = str(salida_funcion)
        
    # Agrego el historial de chat
    messages.append(
        {"role": "tool", "tool_call_id": tc.id, "content": content}
    )

{"name":"Joaquin Lopez"}
{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}


In [67]:
co = cohere.ClientV2()
response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)
print("Respuesta:")
print(response.message.content[0].text)

Respuesta:
El email de Joaquin Lopez es Joacolocolopez@gmail.com.


In [37]:
print(contacts)

{'Joaquin Lopez': {'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}, 'Flavio Oncativo': {'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}}


In [6]:
##Unica celda automatizada
#contiene las funciones necesarias
#tiene definida una lista de tools que se pasa al modelo
#un mensaje donde se pone en contexto al modelo sobre la funcion que debe cumplir
#contiene la funcion procesar_mensaje, la cual recibe una pregunta (str) y retorna una respuesta

In [4]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."

contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Agrega un contacto al diccionario.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto."
                    },
                    "phone": {
                        "type": "string",
                        "description": "Número de teléfono del contacto."
                    },
                    "email": {
                        "type": "string",
                        "description": "Correo electrónico del contacto."
                    },
                },
                "required": ["name", "phone", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Obtiene información específica de un contacto por su nombre, como su número de teléfono o correo electrónico.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto."
                    },
                },
                "required": ["name"]
            }
        }
    }
]

map_funciones = {
    "add_contact": add_contact,
    "get_information": get_information,
} 


mensaje_sistema = """
Eres un asistente que puede administrar contactos utilizando herramientas. 
Si te preguntan por información de un contacto, utiliza la herramienta `get_information` 
para recuperar los datos relacionados. Si te piden agregar un contacto, usa `add_contact`.
Utilizar los nombre tal cual como son ingresados por el usuario, por ejemplo:
Q: Cual es el Email de Joaquin Lopez?" 
tool_plan: Voy a utilizar la herramienta get_information para obtener el email de Joaquin Lopez.
"""



def procesar_mensaje(prompt):
    co = cohere.ClientV2()
    messages = [
        {"role": "system", "content": mensaje_sistema},
        {"role": "user", "content": prompt},
    ]
    
    # Primera llamada a la API
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=messages,
        tools=tools
    )

    # Agrego el historial de chat
    messages.append(
        {
            "role": "assistant",
            "tool_calls": response.message.tool_calls,
            "tool_plan": response.message.tool_plan,
        }
    )
    # Iteramos sobre las tool_calls generadas por el modelo
    for tc in response.message.tool_calls:
        # Llamamos a la función recomendada por el modelo usando los parámetros recomendados
        salida_funcion = map_funciones[tc.function.name](**json.loads(tc.function.arguments))
        
        if isinstance(salida_funcion, dict):
            content = json.dumps(salida_funcion)
        else:
            content = str(salida_funcion)

        # Agregamos el contenido a los mensajes
        messages.append(
            {"role": "tool", "tool_call_id": tc.id, "content": content}
        )
        
    # Segunda llamada para obtener la respuesta final
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=messages,
        tools=tools
    )

    return response.message.content[0].text

In [ ]:
# Prueba de la funcion procesar mensaje enviando una pregunta

In [7]:
respuesta = procesar_mensaje("Cual es el Email de Alvaro?")
print(respuesta)

Lo siento, no he encontrado el email de Alvaro.


## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [ ]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [37]:
co = cohere.ClientV2()

"funcion utilizada para detectar el idioma de la frase que se pase por parametro (string)"
def detector_idioma (frase):
    response_idioma = co.chat(
        model="command-r-plus-08-2024",
        messages=[
            {"role": "system", "content":"Actua como un detector de idiomas profesional, tu tarea es detectar el idioma de entrada del usuario. Responde únicamente con el nombre del idioma, por ejemplo: 'español', 'inglés', 'portugués'."},
            {"role": "user", "content": frase}
        ],
        temperature=0.1,
        seed=42,
    )
    idioma = response_idioma.message.content[0].text
    return idioma

"función utulizada como traductor, el modelo command no daba tan buenos resultados como command-r-plus-08-2024 para esta tarea"
def traductor(frase, idioma_traducir):
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[
            {"role": "system", "content": f"Actua como un traductor profesional, tu tarea es traducir al {idioma_traducir}. Responde solo la traducción y conserva los emojis de la entrada sólo si los hubiera"},
            {"role": "user", "content": frase}
        ],
        temperature=0.1,
        seed=42,
    )

    return response.message.content[0].text


def history_answer(pregunta):
    prompt = f""" 
            ###
            Instrucciones: 
            - Responde la pregunta utilizado el contexto.
            - Responde de manera concisa en solo una oración.
            - Agrega emojis relacionados con la oración al final.
            - Responde en tercera persona.
            - Si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.

            ###
            Contexto:
            {historia}

            ###
            Pregunta:
            {pregunta}

            """

    "llamada al modelo para responder a la pregunta ingresada"
    response_pregunta = co.chat(
        model="command-r-plus-08-2024",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        seed=42,
    )
    
    respuesta = response_pregunta.message.content[0].text

    "llamo a la funcion para detectar los idiomas de la pregunta y la respuesta"
    idioma_pregunta = detector_idioma(pregunta)
    idioma_respuesta = detector_idioma(respuesta)

    "valido que la pregunta y respuesta esten en el mismo idioma, si no realizo la traducción"
    if (idioma_pregunta != idioma_respuesta): 
        respuesta_procesada = traductor(respuesta, idioma_pregunta)
    else:
        respuesta_procesada = respuesta

    
    respuesta_al_usuario = respuesta_procesada + ' Hakuna Matata!'


    
    return respuesta_al_usuario

In [42]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""


pregunta = "¿Cómo se sintió Thomas al cambiar la hoz por la lanza?"


# respuesta
print(history_answer(pregunta))

Thomas sintió una mezcla de miedo y determinación al empuñar la lanza, sabiendo que su vida estaba a punto de cambiar para siempre. 🗡️💪🙏 Hakuna Matata!


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [55]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()
co = cohere.ClientV2()

chat_history = []

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    global chat_history

    chat_history.append({"role": "user", "content": message})

    " no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens."
    if len(chat_history) > 3:
        chat_history = chat_history[-3:]

    prompt = [
        {"role": "system", "content": "Responde de manera positiva, con un tono entusiasta. Responde con consejos útiles, como si fueras un tutor."}
    ] + chat_history
    
    # Aquí puedes conectar tu modelo o lógica de chatbot real
    response = co.chat(
        model="command-r-plus-08-2024",
        messages= prompt,
        max_tokens=70
    )
    
    return response.message.content[0].text

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [57]:
print(chat_history)

[{'role': 'user', 'content': ' Hola, cómo puedo ayudar a alguien en situacion de pobreza?'}, {'role': 'user', 'content': 'Esta persona duerme en la calle'}, {'role': 'user', 'content': 'Puedo ofrecerle trabajo?'}]


### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.

In [13]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.